# Ollama & Private Data

[Ollama](https://ollama.com/) download and install on Mac/Linux/Windows

Available Ollama [models](https://ollama.com/library)
* [Mistral](https://ollama.com/library/mistral)
* Mixtral-8x7B is more powerful and can handle extended conversations - Supported????

Ollama has support for multi-modal LLMs, such as bakllava and llava.


Ollama & [Langchain](https://python.langchain.com/docs/integrations/llms/ollama)


Basic commands:

* Fetch available LLM model via  ```ollama pull <name-of-model>```
* On Mac, the models will be download to ```~/.ollama/models```
* To view all pulled models, use ```ollama list```
* To chat directly with a model from the command line, use ```ollama run <name-of-model>```



--------------------
## References:

* Building a Multi-Document Chatbot Using [Mistral 7B, ChromaDB, and Langchain](https://www.e2enetworks.com/blog/building-a-multi-document-chatbot-using-mistral-7b-chromadb-and-langchain)
* [Ask Your Web Pages Using Mistral-7b & LangChain](https://medium.com/@zekaouinoureddine/ask-your-web-pages-using-mistral-7b-langchain-f976e1e151ca)
* [Ollama Python Library Released! How to implement Ollama RAG?](https://www.youtube.com/watch?v=4HfSfFvLn9Q)
  * Ollama RAG [code](https://mer.vin/2024/01/ollama-rag/)






In [ ]:
#from langchain_community.llms import Ollama
from langchain.llms import Ollama

llm = Ollama(model="mistral")

llm.invoke("Tell me a joke")

------------------------

-----
## Environment set up



In [ ]:
!pip install -qU \
    langchain \
    tiktoken \
    ollama \
    pypdf \
    chromadb \
    pinecone-client \
    ipywidgets \
    langflow 



---------------------
# Mistral Embedding





---------
# Data Ingestion


<img src='./data/data_ingestion.png' width='800'>



--> Point to data source and load multiple documents (PDF/Word/HTML/Chat...). [Document Loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)

--> **Chunk** into smaller parts. [Text Splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)
  * Optimize for the smallest size without losing context
  * Consider adding some meaningful global metadata in all the chunks giving global context to all your embedded chunks
  * Use ```chunk_overlap``` to maintain some local context
  
--> Create **embedding** vectors for each chunk using LLM embedding. [Text Embedding Models](https://python.langchain.com/en/latest/modules/models/text_embedding.html)
  * An embedding is a vector (list) of floating point numbers
  * Embeddings are an AI native way to represent any kind of data: **text, images, audio and video**
        
--> Store embedding + metadata in 
        [Vector stores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

  * **Vector stores**:
    * [Pinecone](https://docs.pinecone.io/docs/overview): Managed vector store. Pinecone vector search index (OpenAI dimension: 1536)
    * [Chroma](https://docs.trychroma.com/): Open source locally managed vector store.
    * [Qdrant](https://github.com/qdrant/qdrant): Open source vectorstore with local and cloud managed options
    * PostgreSQL with [pg_vector](https://github.com/pgvector/pgvector)

    
--> **Semantic search** to retrieve relevant information by measuring the similarity between two vectors.
  * Typical similarity metrics: **Cosine, Dot Product, Euclidean** 
            

-----
## Load documents and chunk



In [ ]:
# Setting some variable used global for the following cells
import os

persist_chroma_directory = '.chroma_db'
pdf_folder = './data/pdf'

os.listdir(pdf_folder)


In [ ]:
from langchain.document_loaders import DirectoryLoader, \
                                        PyPDFLoader, \
                                        UnstructuredPDFLoader, \
                                        TextLoader

loader = DirectoryLoader(pdf_folder, glob='**/*.pdf', loader_cls=PyPDFLoader)
documents = loader.load()

# If using PyPDFLoader each document in documents is 1 page of a pdf. 
print(f'{len(documents)} pages loaded')


In [ ]:
documents[0]

In [ ]:
print(documents[0].page_content)

----
## Split in to smaller chunks

* Split the text up into small, semantically meaningful chunks.
* Most LLMs are constrained by the number of tokens that you can pass in so passing in an entire document or several document pages + prompt may exceed LLM token limit



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chunk loaded documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f'{len(chunks)} chunks created')

In [ ]:
chunks[:2]


In [ ]:
print(chunks[0].page_content)

----
## Chroma: Create document embeddings

[Chroma](https://docs.trychroma.com/): Open source locally managed vector store.

Mistral [embedding pricing](https://docs.mistral.ai/platform/pricing/)


In [ ]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

persist_chroma_directory = '.chroma_db'


#embedding = MistralAIEmbeddings(mistral_api_key="your-api-key")
#embedding.model = "mistral-embed"  # or your preferred model if available
# use OpenAI embedding
#embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, \
#                             model='text-embedding-ada-002')


embeddings = OllamaEmbeddings(model="mistral")
chroma_store = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=persist_chroma_directory)

# Persist the database --> Need to call persist() when using Jupyter
chroma_store.persist()
chroma_store = None




----------------
# Retrieval Augmented Generation (RAG)


<img src='./data/RAG2.png' width='1000'>

RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

The idea of [Retrieval Augmented Generation (RAG)](https://huggingface.co/docs/transformers/model_doc/rag) workflow is simple. Instead of asking a question directly, the process first uses the user question to perform a search to retrieve relevant documents from the internal dataset and then provides these documents together with the question to LLM. With the additional context the LLM can answer as though it has been trained with the internal dataset.



In [10]:
import ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

persist_chroma_directory = '.chroma_db'
embeddings = OllamaEmbeddings(model="mistral")
chroma_store = Chroma(embedding_function=embeddings, persist_directory=persist_chroma_directory)


retriever = chroma_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
#query = "Provide details of compensation if my flight is cancelled? "
query = "Provide details of compensation if my flight is cancelled? Output the results in bullet points"
#query = "How much liquid can I bring on a flight?"
#query = "how long is my ticket valid for?"

result = rag_chain(query)

print(result)


 Compensation for Flight Cancellations by Aer Lingus (as per Article 3.AER LINGUS DELAY NOTICE):

- Applicability:
  * For flights departing from an EU airport or a third country to an EU airport with Aer Lingus as the operating carrier.
  * Conditions: You have a confirmed reservation, present yourself for check-in on time, and are travelling at a publicly available fare.

Rules for Assistance when a Flight is Cancelled:

- When reasonably expected departure time exceeds:
  * Two hours for flights of 1500 kilometres or less.
  * Three hours for intra-Community flights over 1500 kilometres and other flights between 1500-3500 kilometres.
  * Four hours for all other flights.
- Free assistance:
  * Meals and refreshments proportional to waiting time.
  * Two telephone calls, telex or fax messages, or e-mails.
- Additional assistance when departure time is at least a day later than originally announced:
  * Hotel accommodation if a stay of one or more nights becomes necessary or additiona

-------------------------
# Ollama RAG UI

https://mer.vin/2024/01/ollama-rag/


